## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

sys.path.append("../")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)





In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,Cit_ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
0,0,Vestmannaeyjar,IS,63.4427,-20.2734,33.35,71,100,9.75,overcast clouds,22-02-11 23:44:14
1,1,Severo-Kurilsk,RU,50.6789,156.1250,26.83,83,99,9.33,light snow,22-02-11 23:47:19
2,2,Kapaa,US,22.0752,-159.3190,82.47,63,0,1.99,clear sky,22-02-11 23:47:19
3,3,Ilulissat,GL,69.2167,-51.1000,26.62,74,75,11.50,broken clouds,22-02-11 23:44:51
4,4,Ponta Do Sol,PT,32.6667,-17.1000,64.06,75,27,6.69,scattered clouds,22-02-11 23:47:20


In [23]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_temp_criteria = float(input("What is the highest daily high temp you would want to experience on a trip?"))
min_temp_criteria = float(input("What is the lowest daily high temp you would want to experience on a trip?"))

In [24]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
trip_criteria_df = city_data_df.loc[(city_data_df["Max Temp"]>=min_temp_criteria) & (city_data_df["Max Temp"]<= max_temp_criteria)]
trip_criteria_df.head(10)

,Cit_ID,City,Country,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Description,Date
4,4,Ponta Do Sol,PT,32.6667,-17.1000,64.06,75,27,6.69,scattered clouds,22-02-11 23:47:20
17,17,Punta Arenas,CL,-53.1500,-70.9167,55.51,71,75,32.21,broken clouds,22-02-11 23:47:27
25,25,Kaduna,NG,10.5222,7.4383,64.94,15,4,8.93,clear sky,22-02-11 23:47:31
32,32,Ribeira Grande,PT,38.5167,-28.7000,60.12,73,100,10.85,overcast clouds,22-02-11 23:43:31
39,39,Hobart,AU,-42.8794,147.3294,64.22,62,46,4.29,scattered clouds,22-02-11 23:47:33
49,49,Mataura,NZ,-46.1927,168.8643,58.48,54,90,9.98,overcast clouds,22-02-11 23:47:43
54,54,New Norfolk,AU,-42.7826,147.0587,63.91,48,19,3.00,few clouds,22-02-11 23:44:05
78,78,Lebu,CL,-37.6167,-73.6500,59.00,76,0,23.06,clear sky,22-02-11 23:48:42
83,83,Kaitangata,NZ,-46.2817,169.8464,57.92,65,86,6.33,overcast clouds,22-02-11 23:48:45
92,92,Hermanus,ZA,-34.4187,19.2345,64.90,89,1,7.70,clear sky,22-02-11 23:48:51


In [25]:
# 4a. Determine if there are any empty rows.
trip_criteria_df.isnull().sum()

Cit_ID         0
City           0
Country        0
Lat            0
Long           0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
Date           0
dtype: int64

In [26]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
trip_criteria_df = trip_criteria_df.dropna()

In [28]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = trip_criteria_df[["City", "Country", "Max Temp", "Description", "Lat", "Long"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head()
# hotel_df.count()

,City,Country,Max Temp,Description,Lat,Long,Hotel Name
4,Ponta Do Sol,PT,64.06,scattered clouds,32.6667,-17.1000,
17,Punta Arenas,CL,55.51,broken clouds,-53.1500,-70.9167,
25,Kaduna,NG,64.94,clear sky,10.5222,7.4383,
32,Ribeira Grande,PT,60.12,overcast clouds,38.5167,-28.7000,
39,Hobart,AU,64.22,scattered clouds,-42.8794,147.3294,


In [29]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    long = row["Long"]
    locations = [lat,long]
    params["location"]=f"{lat},{long}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    print(f"Retrieving Results for INdex {index}.")
    hotels = requests.get(base_url, params=params).json()
    results = hotels["results"]
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index,"Hotel Name"] = hotels['results'][0]['name']
    except (IndexError):
        print("Hotel not nearby... skipped.")
        

Retrieving Results for INdex 4.
Retrieving Results for INdex 17.
Retrieving Results for INdex 25.
Retrieving Results for INdex 32.
Retrieving Results for INdex 39.
Retrieving Results for INdex 49.
Retrieving Results for INdex 54.
Retrieving Results for INdex 78.
Retrieving Results for INdex 83.
Retrieving Results for INdex 92.
Retrieving Results for INdex 94.
Hotel not nearby... skipped.
Retrieving Results for INdex 95.
Retrieving Results for INdex 100.
Retrieving Results for INdex 101.
Retrieving Results for INdex 134.
Retrieving Results for INdex 137.
Retrieving Results for INdex 144.
Retrieving Results for INdex 157.
Retrieving Results for INdex 170.
Retrieving Results for INdex 178.
Retrieving Results for INdex 181.
Retrieving Results for INdex 191.
Retrieving Results for INdex 195.
Retrieving Results for INdex 202.
Retrieving Results for INdex 217.
Retrieving Results for INdex 237.
Retrieving Results for INdex 245.
Retrieving Results for INdex 258.
Retrieving Results for INdex 265

In [32]:
# bring in numpy
import numpy as np

In [35]:
# 7. Drop the rows where there is no Hotel Name.
# hotel_df
cleaned_hotel_df = hotel_df.replace('', np.nan)
cleaned_hotel_df = cleaned_hotel_df.dropna()
cleaned_hotel_df

,City,Country,Max Temp,Description,Lat,Long,Hotel Name
4,Ponta Do Sol,PT,64.06,scattered clouds,32.6667,-17.1000,Hotel do Campo
17,Punta Arenas,CL,55.51,broken clouds,-53.1500,-70.9167,Hotel Hain
25,Kaduna,NG,64.94,clear sky,10.5222,7.4383,HAMDALA HOTEL
32,Ribeira Grande,PT,60.12,overcast clouds,38.5167,-28.7000,Quinta da Meia Eira
39,Hobart,AU,64.22,scattered clouds,-42.8794,147.3294,Mantra on Collins Hobart
...,...,...,...,...,...,...,...
627,Ziway,ET,63.64,broken clouds,7.9333,38.7167,Haile Resort Ziway Batu
629,Tuscaloosa,US,61.23,clear sky,33.2098,-87.5692,Hampton Inn Tuscaloosa-University
653,Estelle,US,64.00,clear sky,29.8458,-90.1067,Boomtown Casino & Hotel New Orleans
662,Barmer,IN,58.77,clear sky,25.7500,71.3833,OYO 71911 Hotel Amber Palace


In [37]:
# 8a. Create the output File (CSV)
output_cleaned_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
cleaned_hotel_df.to_csv(output_cleaned_data_file, index_label="City_ID")

In [39]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description: </dt><dd>{Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in cleaned_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = cleaned_hotel_df[["Lat", "Long"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 2.0)
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))